In [770]:
import math
import pymysql
import pandas as pd


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import wikipedia
import spacy
from textblob import TextBlob
from IPython.display import Image

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [771]:
restaurants_df = pd.read_csv('https://raw.githubusercontent.com/Ajinth/Recommender-System/master/Research/Sample_Restaurants.csv', index_col='Users')

In [772]:
restaurants_df.fillna(value=0, inplace=True)

In [773]:
restaurants_df.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0


# Collaborative Item to Item Filtering

In [774]:
'''List of All Restaurants from the Dataframe'''
all_restaurants = list(restaurants_df.columns)
all_restaurants

['Little_Donkey',
 'Caf__Nero',
 'India_Pavilion',
 'Desi_Dhaba',
 'Thai_Sensation',
 'Dumpling_House ',
 'Caf__Luna']

In [775]:
# '''List of Restaurants for Ajinth'''
# ajinth_restaurant_list = list(restaurants_df.loc['Ajinth', :] ==0)
# ajinth_restaurant_list

In [776]:
# '''List of Restaurants that Ajinth has Rated'''
# user_rated=[]
# for key, a_res in enumerate(ajinth_restaurant_list):
#     if a_res == False:
#         user_rated.append((restaurants_df.loc['Ajinth', :] ==0).index[key])
#     else: 
#         continue
# print (user_rated)  

In [777]:
'''Function to return the list of restaurants rated by the user'''
def user_rated_list(user):
    user_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == False:
            user_rated.append((restaurants_df.loc[user, :] ==0).index[key])
        else: 
            continue
    return user_rated   

In [778]:
user = 'Tess'
user_rated = user_rated_list(user)
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [779]:
# '''List of Restaurants that Ajinth has Not Rated'''
# user_not_rated=[]
# for key, a_res in enumerate(ajinth_restaurant_list):
#     if a_res == True:
#         user_not_rated.append((restaurants_df.loc['Ajinth', :] ==0).index[key])
#     else: 
#         continue
# print (user_not_rated) 

In [780]:
'''Function to return the list of restaurants yet to be rated by the user'''
def user_not_rated_list(user):
    user_not_rated=[]
    for key, a_res in enumerate(list(restaurants_df.loc[user, :] ==0)):
        if a_res == True:
            user_not_rated.append((restaurants_df.loc['Ajinth', :] ==0).index[key])
        else: 
            continue
    return (user_not_rated) 

In [781]:
user = 'Tess'
user_not_rated = user_not_rated_list(user)
user_not_rated

['Desi_Dhaba', 'Dumpling_House ', 'Caf__Luna']

In [782]:
# (restaurants_df.loc['Ajinth', :] ==0).index[0]

In [783]:
'''Custom Function to calculate the Co-Similarity'''
def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [784]:
# '''
# Looping through the User Restaurants and all restaurants to calculate the Matrix.
# The results are stored in a dictionary
# '''
# ur_rest_dict = {}
# for ur in user_rated: 
#     ur_cosim=[]
#     for ar in all_restaurants: 
#         similarity = cosim(restaurants_df[ur],\
#                                         restaurants_df[ar])
#         ur_cosim.append(similarity)
#     ur_rest_dict[ur] = ur_cosim
# # print (ur_rest_dict)

In [785]:
'''This function calcultes the co-sine similarity between the restaurants rsted by the user and all the restaurants in the list'''
def cooccurence_matrix_creator():
    '''Calling the user rated function'''
    user_rated_list(user)
    
    ur_rest_dict = {}
    for ur in user_rated: 
        ur_cosim=[]
        for ar in all_restaurants: 
            similarity = cosim(restaurants_df[ur],\
                                            restaurants_df[ar])
            ur_cosim.append(similarity)
        ur_rest_dict[ur] = ur_cosim
    co_occurence_matrix = pd.DataFrame.from_dict(ur_rest_dict, orient='index')
    co_occurence_matrix.columns=all_restaurants
    return co_occurence_matrix

In [786]:
co_occurence_matrix = cooccurence_matrix_creator()
co_occurence_matrix.head()

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


In [787]:
# '''Dataframe for the co-occurence matrix'''
# co_occurence_matrix = pd.DataFrame.from_dict(ur_rest_dict, orient='index')
# co_occurence_matrix.columns=all_restaurants
# co_occurence_matrix

# Review of some of the Variables

In [788]:
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [789]:
user_not_rated

['Desi_Dhaba', 'Dumpling_House ', 'Caf__Luna']

In [790]:
co_occurence_matrix

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Little_Donkey,1.000000,0.606349,0.811778,0.00000,0.862105,0.309839,0.651273
Caf__Nero,0.606349,1.000000,0.170910,0.29173,0.272259,0.782794,0.174034
India_Pavilion,0.811778,0.170910,1.000000,0.00000,0.904497,0.000000,0.706049
Thai_Sensation,0.862105,0.272259,0.904497,0.00000,1.000000,0.000000,0.809807


In [791]:
restaurants_df

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


In [792]:
# type(restaurants_df.loc['Ajinth', : ])

In [793]:
# restaurants_to_rate = co_occurence_matrix[user_not_rated_list(user)]
# restaurants_to_rate

In [794]:
def rest_to_rate(df):
    restaurants_to_rate = df[user_not_rated_list(user)]
    return restaurants_to_rate

In [795]:
restaurants_to_rate = rest_to_rate(co_occurence_matrix)
restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Little_Donkey,0.00000,0.309839,0.651273
Caf__Nero,0.29173,0.782794,0.174034
India_Pavilion,0.00000,0.000000,0.706049
Thai_Sensation,0.00000,0.000000,0.809807


In [796]:
'''Generating the prediction'''


def rating_predictor_colab_ii():
    predicted_rating_dict = {}
    for not_rated in restaurants_to_rate.columns: 
        column_df = pd.DataFrame(restaurants_to_rate[not_rated].sort_values(ascending=False))
        predicted_rating = []
        top_sim_list = []
        for top_n in range(2):   # Just getting the top two rated items 

            top_sim = column_df[not_rated][top_n]
            top_sim_list.append(top_sim)

            given_rr = column_df[not_rated].index[top_n]
            given_rank_df = pd.DataFrame(restaurants_df.loc[user, :]) # While converting this to function the name should be an argument to the function
            given_rank = given_rank_df.loc[given_rr,:]

            predicted_rating_item = float(given_rank*top_sim)
            predicted_rating.append(predicted_rating_item)
        total_sim_weight = sum(top_sim_list)
        adjusted_predicted_rating = (sum(predicted_rating))/total_sim_weight
        predicted_rating_dict[not_rated] = adjusted_predicted_rating
        
    '''Creating the Dataframe in this step'''
        
    predicted_df = pd.DataFrame.from_dict(predicted_rating_dict, orient='index')
    predicted_df.columns=[user]
    return predicted_df
#     return (predicted_rating_dict)


In [797]:
predicted_df = rating_predictor_colab_ii()
predicted_df.head()

,Tess
Desi_Dhaba,3.000000
Dumpling_House,3.000000
Caf__Luna,3.534224


# Collaborative User to User Filtering

In [798]:
restaurants_df

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


In [799]:
user_rated

['Little_Donkey', 'Caf__Nero', 'India_Pavilion', 'Thai_Sensation']

In [800]:
user_not_rated

['Desi_Dhaba', 'Dumpling_House ', 'Caf__Luna']

In [801]:
'''This function calcultes the co-sine similarity between different users for the user-user collaborative filtering model'''

def user_cooccurence_matrix_creator():
    user_dict = {}
    user_columns = []
    user_cosim=[]
    
    for ur in restaurants_df.iterrows(): 
        if ur[0] == user: 
            continue 
        else: 
            user_similarity = cosim(restaurants_df.loc[ur[0],:].values, restaurants_df.loc[user,:].values)
            user_cosim.append(user_similarity)
            user_columns.append(ur[0])
    user_dict[user] = user_cosim
    
    user_df = pd.DataFrame.from_dict(user_dict, orient='index')
    user_df.columns = user_columns
    return user_df
    


In [802]:
ur_coocurence_df = user_cooccurence_matrix_creator()
ur_coocurence_df

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [803]:
'''Sorting the dataframe here in descending order'''
ur_coocurence_df.sort_index(ascending=False, axis=0, inplace=True)
ur_coocurence_df

,Ajinth,Jessica,Mark,Joe,Chris
Tess,0.809345,0.807427,0.172917,0.708676,0.517597


In [804]:
restaurants_df.loc[:,:]

,Little_Donkey,Caf__Nero,India_Pavilion,Desi_Dhaba,Thai_Sensation,Dumpling_House,Caf__Luna
Users,,,,,,,
Ajinth,4.0,0.0,5.0,0.0,3.0,0.0,2.0
Tess,3.0,3.0,3.0,0.0,4.0,0.0,0.0
Jessica,4.0,3.0,0.0,0.0,2.0,0.0,1.0
Mark,0.0,2.0,0.0,2.0,0.0,2.0,4.0
Joe,5.0,0.0,5.0,0.0,8.0,0.0,8.0
Chris,3.0,5.0,0.0,0.0,0.0,4.0,0.0


In [805]:
'''This cell should be deleted after testing. The function already defined once'''
def rest_to_rate(df):
    restaurants_to_rate = df[user_not_rated_list(user)]
    return restaurants_to_rate

In [806]:
ur_restaurants_to_rate = rest_to_rate(restaurants_df)
ur_restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Tess,0.0,0.0,0.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [807]:
'''Test Code to loop through the Co-Occurence Matrix and select the top 2 similar users'''
counter = 0 
for i in ur_coocurence_df: 
    if counter < 2:
        print (i, float(ur_coocurence_df[i]))
        counter += 1
    else: 
        break

Ajinth 0.8093446482740974
Jessica  0.8074267672650716


In [808]:
'''Making updates to the loop'''
counter = 0
for i in ur_coocurence_df: 
    if counter < 2:
#         print (i, float(ur_coocurence_df[i]))
        user = i 
        user_similarity = float(ur_coocurence_df[i]) 
        counter += 1
    else: 
        break

In [809]:
counter = 0 
for i in ur_coocurence_df: 
    if counter < 2:
        user = i 
        print (user)
        user_similarity = float(ur_coocurence_df[i]) 
        print (user_similarity)
        counter += 1
        
    else: 
        break

Ajinth
0.8093446482740974
Jessica 
0.8074267672650716


In [810]:
topn_user_df = restaurants_df.loc[['Tess', 'Jessica '], user_not_rated ]
topn_user_df

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Tess,0.0,0.0,0.0
Jessica,0.0,0.0,1.0


In [811]:
for item in topn_user_df: 
    print (item)
#     print (topn_user_df[item][0])

    

Desi_Dhaba
Dumpling_House 
Caf__Luna


In [812]:
ur_coocurence_df_t= ur_coocurence_df.T
ur_coocurence_df_t.columns=['Similarity']
ur_coocurence_df_t

,Similarity
Ajinth,0.809345
Jessica,0.807427
Mark,0.172917
Joe,0.708676
Chris,0.517597


In [813]:
ur_restaurants_to_rate.drop('Tess', axis = 0, inplace=True )
ur_restaurants_to_rate

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [814]:
joint_matrix = ur_restaurants_to_rate.join(ur_coocurence_df_t, how='inner')
joint_df = joint_matrix.iloc[:2,:]
joint_df

,Desi_Dhaba,Dumpling_House,Caf__Luna,Similarity
Users,,,,
Ajinth,0.0,0.0,2.0,0.809345
Jessica,0.0,0.0,1.0,0.807427


In [815]:
for col in joint_df: 
    print 

In [816]:
ur_restaurants_to_rate

,Desi_Dhaba,Dumpling_House,Caf__Luna
Users,,,
Ajinth,0.0,0.0,2.0
Jessica,0.0,0.0,1.0
Mark,2.0,2.0,4.0
Joe,0.0,0.0,8.0
Chris,0.0,4.0,0.0


In [817]:
ur_coocurence_df_t

,Similarity
Ajinth,0.809345
Jessica,0.807427
Mark,0.172917
Joe,0.708676
Chris,0.517597


In [818]:
predicted_rating_df = ur_restaurants_to_rate.mul(ur_coocurence_df_t,axis=0,)
predicted_rating_df

,Caf__Luna,Desi_Dhaba,Dumpling_House,Similarity
Users,,,,
Ajinth,NaN,NaN,NaN,NaN
Jessica,NaN,NaN,NaN,NaN
Mark,NaN,NaN,NaN,NaN
Joe,NaN,NaN,NaN,NaN
Chris,NaN,NaN,NaN,NaN
